# Exporting NNP models for GROMACS
In this example, we want to go over how to export models trained in Pytorch for use with the NNP interface in GROMACS. 

To learn how to wrap models so that they're compatible with the interface, look at some of the examples in the `models` folder. In general, they only need to define a forward function conforming to the following requirements:
1. The inputs should consist only of tensors representing one or several of the options given by the interface (e.g., atom positions, atomic numbers, simulation box vectors, PBCs, etc.). Remember that in GROMACS, inputs are passed to the model in order of their occurence in the `.mdp` file, so the order should match. 
2. The model should return a tensor containing the total energy of the NNP system. By default, GROMACS will then calculate the forces as negative gradients of the energy tensor w.r.t the _first_ input, which should be the positions. Optionally, these can be overriden by returning an additional force tensor, which is useful if the forces have to be calculated in a non-standard way.

## ANI models
First, let's export some pre-trained models from the [TorchANI package](https://github.com/aiqm/torchani). When you first use these models, the package will automatically download the pre-trained model weights from the Github repository. We'll use the ANI2x version, with the full ensemble. If you only want to use a single model, you can specify a model index from 0 to 7. This will speed up the calculation, but lead to less accurate results. 

In [ ]:
import torch
from models.ani import GmxANIModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'

save_path = 'models/ani2x.pt'
model = GmxANIModel(version=2, device=device)
torch.jit.script(model).save(save_path)

### CUAEV
To use TorchANI's CUDA extension for the calculation of the AEVs, we have to tell the Libtorch version used by GROMACS about the extension. You can follow the instructions in the [GROMACS install guide](https://manual.gromacs.org/2025.0/install-guide/index.html#building-with-neural-network-potential-support), or alternatively, we can export the path to the extension with the model itself, so that the extension can be loaded at runtime.

In [ ]:
save_path = 'models/ani2x_cuaev.pt'
model = GmxANIModel(use_opt='cuaev', version=2, device=device)

# cuaev doesn't seem to be registered as a proper extension library
# it's usually found in the site-packages directory under the torchani package
ext_lib = "~/anaconda3/envs/nnpot/lib/python3.12/site-packages/torchani/cuaev.cpython-312-x86_64-linux-gnu.so"
extra_files = {}
extra_files['extension_libs'] = ext_lib

torch.jit.script(model).save(save_path, _extra_files=extra_files)

### NNPOps
To use the highly optimized NNPOps package to accelerate the inference even further, we similarly have to tell Libtorch about the extension. Another special feature is that NNPOps optimizations rely on static CUDA graphs, which is why the exact form of the model needs to be known at compile-time. Therefore, we need to specify the atomic numbers used in the model. To match the sequence passed to the model by GROMACS, check the `.gro` file of the simulation.

In [ ]:
save_path = 'models/ani2x_nnpops.pt'

# example atomic number tensor for alanine dipeptide
atomic_numbers = torch.tensor([1,6,1,1,6,8,7,1,6,1,6,1,1,1,6,8,7,1,6,1,1,1], device=device)
model = GmxANIModel(use_opt='nnpops', atomic_numbers=atomic_numbers, version=2, device=device)

# nnpops can be found by checking for torch extension library
ext_lib = []
for lib in torch.ops.loaded_libraries:
    if lib:
        ext_lib.append(lib)
# if multiple extensions are found, they are separated by ':'
ext_lib = ":".join(ext_lib)
print("loaded extension libraries: ", ext_lib)
extra_files = {}
extra_files['extension_libs'] = ext_lib

torch.jit.script(model).save(save_path, _extra_files=extra_files)